In [13]:
import math
import os
from glob import glob
from itertools import combinations
from urllib.error import URLError

import earthpy as et
import earthpy.appeears as etapp
import geopandas as gpd
import holoviews as hv
import hvplot as hv
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import requests
import rasterio
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from IPython.display import Image
import utm
import xarray as xr
import xrspatial
import warnings
import zipfile

In [7]:
# Set up directory
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME, 'treebeard')
# Create the directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

las_index_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'lidar_index_cspn_q2',
    'lidar_index_cspn_q2.shp'
)

# Download LIDAR index tiles
if not os.path.exists(las_index_path):
    las_index_url = ('https://gisdata.drcog.org:8443/geoserver/DRCOGPUB/'
             'ows?service=WFS&version=1.0.0&request=GetFeature&'
             'typeName=DRCOGPUB:lidar_index_cspn_q2&outputFormat=SHAPE-ZIP')

    las_index_shp = et.data.get_data(url=index_url)

las_index_gdf = (
    gpd.read_file(las_index_path).to_crs(4326)
    #.loc[['Thunder Basin National Grassland', 'Pawnee National Grassland']]
)

las_index_plot = las_index_gdf.hvplot(
            geo=True, 
            tiles='EsriNatGeo',
            aspect='equal'
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [42]:
# Download test DRAPP tile and use it to find overlapping LAS files

test_tile_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'N3W345.tif'
)

if not os.path.exists(test_tile_path):
    test_tile_url = 'https://drapparchive.s3.amazonaws.com/2020/N3W345.tif'
    et.data.get_data(url=test_tile_url)
    
test_aerial = rxr.open_rasterio(test_tile_path)

test_aerial = test_aerial.rio.reproject(las_index_gdf.crs)

#aerial_bounds = test_aerial.rio.bounds()

test_aerial_plot = test_aerial.hvplot.rgb(rasterize=True, aspect='equal')
test_aerial_plot

Invoked as dynamic_operation(1)
Invoked as dynamic_operation(1, height=400, scale=1.0, width=400, x_range=None, y_range=None)
Invoked as dynamic_operation(1, height=400, scale=1.0, width=400, x_range=None, y_range=None)


DataError: Key dimension values and value array R shapes do not match. Expected shape (4550, 5926), actual shape: (1, 4550)

ImageInterface expects gridded data, for more information on supported datatypes see http://holoviews.org/user_guide/Gridded_Datasets.html

:DynamicMap   [band]